<a href="https://colab.research.google.com/github/Rodiyaturrahmi/analisis_sentimen_svm/blob/main/analisis_sentimen_svm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory #import Indonesian Stemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 8.1 MB/s 


Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset = 'drive/MyDrive/sentimen/dataset_tweet_sentimen.csv'
data = pd.read_csv(dataset)
data

,tweet,Sentimen
0,https://bit.ly/2L6CcbB | Seharusnya saat ini...,Negatif
1,Klo kata gw Pemerintah tuh lagi menerapkan Her...,Negatif
2,Saat ini yang bisa saya lakukan hanya menyiapk...,Negatif
3,"emosi bgt, lg kondisi begini gue disuruh liput...",Negatif
4,Bang gimna pemerintah mau peduli RRC urus ABK ...,Negatif
...,...,...
995,"Kalian tak percaya Covid ada, himbauan pemerin...",Positif
996,"Berat perjuangan pemerintah Aceh, ketika rakya...",Positif
997,Anjuran pemerintah tidak boleh berpegangan tan...,Positif
998,"gue bingung data covid naik terus, yang salah ...",Positif


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
data['tweet'][1]

'Klo kata gw Pemerintah tuh lagi menerapkan Herd immunity karena kemampuan pemerintah nihil soal pemberantasan covid. Gambling udah siapa kuat dia bertahan hidup yang lemah maka mati. Padahal dinegara lain begitu serius menangani virus tsb.'

In [ ]:
lemma = WordNetLemmatizer()
stemmer = PorterStemmer()
stop_words = set(stopwords.words('indonesian','english'))                       #
def Cleantweet(txt):
  txt = re.sub(r'http\s+', ' ', txt)
  txt = re.sub('[^a-zA-Z]', ' ', txt)
  txt = str(txt).lower()
  txt = word_tokenize(txt)
  txt = [item for item in txt if item not in stop_words]
  txt = [stemmer.stem(i) for i in txt]
  # txt = [lemma.lemmatize(word=w,pos='v') for w in txt]
  txt = [i for i in txt if len(i) > 2]
  txt = ' '.join(txt)
  return txt
data['Cleantweet'] = data['tweet'].apply(Cleantweet)

In [ ]:
data

,tweet,Sentimen,Cleantweet
0,https://bit.ly/2L6CcbB | Seharusnya saat ini...,Negatif,http bit ccbb pemerintah lampung bantuan mahas...
1,Klo kata gw Pemerintah tuh lagi menerapkan Her...,Negatif,klo pemerintah tuh menerapkan herd immun kemam...
2,Saat ini yang bisa saya lakukan hanya menyiapk...,Negatif,lakukan makanan berbuka sahur kepala keluarga ...
3,"emosi bgt, lg kondisi begini gue disuruh liput...",Negatif,emosi bgt kondisi gue disuruh liputan covid do...
4,Bang gimna pemerintah mau peduli RRC urus ABK ...,Negatif,bang gimna pemerintah peduli rrc uru abk menin...
...,...,...,...
995,"Kalian tak percaya Covid ada, himbauan pemerin...",Positif,percaya covid himbauan pemerintah diabaikan br...
996,"Berat perjuangan pemerintah Aceh, ketika rakya...",Positif,berat perjuangan pemerintah aceh rakyatnya per...
997,Anjuran pemerintah tidak boleh berpegangan tan...,Positif,anjuran pemerintah berpegangan tangan mencegah...
998,"gue bingung data covid naik terus, yang salah ...",Positif,gue bingung data covid salah pemerintah masyar...


Perform SVM

In [ ]:
# split x dan y
x = data['Cleantweet']
y = data['Sentimen']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
x_test

695    pemerintah aceh berhasil intervensi aspek kehi...
290    sembilan sepuluh pemerintah indonesia takkan m...
512    puse sejalan pemerintah pusat selesainya covid...
411    bingung pemerintah data covid terinfeksi posit...
403    salam membara ditengah pandemik covid menyelen...
                             ...                        
379    viru covid berlanjut psbb terapkan daerah keme...
756    penerapan psbb melulu penindakan keikitsertaan...
943    pandemi covid diambang gari merah meruntuhkan ...
368    mari kembalikan penanggulan wabah covid pemeri...
820    maaf who kayaknya salah paham sedari dipersiap...
Name: Cleantweet, Length: 200, dtype: object

In [ ]:
# perform count vectorizer
vectorizer = CountVectorizer()
vectorizer.fit(x_train)

CountVectorizer()

In [ ]:
# x_train
x_train = vectorizer.transform(x_train)
x_test = vectorizer.transform(x_test)

In [ ]:
x_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
from sklearn import svm
#Create a svm Classifier
clf = svm.SVC(kernel='linear') 
# Linear Kernel#Train the model using the training sets
clf.fit(x_train, y_train) 
#Predict the response for test dataset
y_pred = clf.predict(x_test)

In [ ]:
y_pred

array(['Negatif', 'Negatif', 'Negatif', 'Negatif', 'Positif', 'Negatif',
       'Positif', 'Positif', 'Negatif', 'Positif', 'Positif', 'Negatif',
       'Positif', 'Negatif', 'Negatif', 'Negatif', 'Negatif', 'Positif',
       'Negatif', 'Negatif', 'Positif', 'Positif', 'Negatif', 'Negatif',
       'Negatif', 'Negatif', 'Negatif', 'Negatif', 'Positif', 'Positif',
       'Negatif', 'Positif', 'Negatif', 'Positif', 'Negatif', 'Negatif',
       'Negatif', 'Positif', 'Negatif', 'Negatif', 'Negatif', 'Negatif',
       'Negatif', 'Positif', 'Negatif', 'Negatif', 'Negatif', 'Negatif',
       'Negatif', 'Positif', 'Positif', 'Negatif', 'Positif', 'Positif',
       'Positif', 'Negatif', 'Negatif', 'Negatif', 'Positif', 'Negatif',
       'Negatif', 'Positif', 'Negatif', 'Positif', 'Positif', 'Negatif',
       'Positif', 'Positif', 'Negatif', 'Negatif', 'Negatif', 'Negatif',
       'Negatif', 'Negatif', 'Negatif', 'Positif', 'Negatif', 'Negatif',
       'Negatif', 'Negatif', 'Negatif', 'Negatif', 

In [ ]:
y_test

695    Positif
290    Negatif
512    Negatif
411    Negatif
403    Positif
        ...   
379    Negatif
756    Positif
943    Negatif
368    Positif
820    Negatif
Name: Sentimen, Length: 200, dtype: object

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[81, 16],
       [36, 67]])

In [ ]:
y_pred = clf.predict(x_test)
print ('Accuracy of SVM classifier on test set: {:.2f}' .format(clf.score(x_test, y_test)))
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)
print(classification_report(y_test, y_pred))

Accuracy of SVM classifier on test set: 0.74
[[81 16]
 [36 67]]
              precision    recall  f1-score   support

     Negatif       0.69      0.84      0.76        97
     Positif       0.81      0.65      0.72       103

    accuracy                           0.74       200
   macro avg       0.75      0.74      0.74       200
weighted avg       0.75      0.74      0.74       200

